In [1]:
import spacy
import pandas as pd
from rapidfuzz import process, fuzz
import re

In [14]:
import pandas as pd
import re
from fuzzywuzzy import fuzz, process
import spacy

# Cargar el DataFrame original con entidades
df = pd.read_csv("df_con_entidades_por_etiqueta_TEST.csv")

# Cargar el DataFrame con las colonias conocidas (nombre y geometría)
df_colonias = pd.read_csv("Colonias_Filtradas.csv")

# Función para normalizar el prefijo y eliminar puntos
def normalizar_prefijos(entidad):
    entidad = entidad.lower()  # Ignorar mayúsculas/minúsculas
    entidad = entidad.replace(".", "")  # Eliminar puntos

    # Reemplazar prefijos solo si están al inicio
    entidad = re.sub(r"^av( |$)", "avenida ", entidad)  # Reemplaza 'av' o 'ave' al inicio
    entidad = re.sub(r"^ave( |$)", "avenida ", entidad)
    entidad = re.sub(r"^col( |$)", "", entidad)  # Elimina el prefijo 'col' o 'colonia'
    entidad = re.sub(r"^colonia( |$)", "", entidad)
    
    return entidad.strip()  # Eliminar espacios en blanco innecesarios

# Función para encontrar la mejor coincidencia y porcentaje
def encontrar_mejor_coincidencia(entidad, df_colonias):
    # Normalizar los nombres en la base de datos antes de comparar
    nombres_colonias_normalizados = [normalizar_prefijos(nombre) for nombre in df_colonias['SETT_NAME']]
    
    # Encontrar la mejor coincidencia y el puntaje
    mejor_coincidencia, score = process.extractOne(entidad, nombres_colonias_normalizados, scorer=fuzz.token_sort_ratio)
    
    # Encontrar el índice de la mejor coincidencia
    idx = nombres_colonias_normalizados.index(mejor_coincidencia)
    
    return mejor_coincidencia, score, idx

# Crear un nuevo DataFrame para almacenar las coincidencias válidas
df_actualizado = pd.DataFrame(columns=df.columns)

# Iterar sobre cada fila de la columna 'colonia'
for idx, colonia in df['colonia'].items():
    if pd.isna(colonia):  # Saltar filas vacías
        continue
    
    # Normalizar el nombre de la colonia antes de comparar
    colonia_normalizada = normalizar_prefijos(colonia)
    
    # Encontrar la mejor coincidencia con la base de datos conocida
    mejor_coincidencia, score, idx_colonia = encontrar_mejor_coincidencia(colonia_normalizada, df_colonias)
    
    # Si la similitud es mayor al 75%, sobrescribir la colonia y agregar la geometría
    if score > 75:
        df.loc[idx, 'colonia'] = mejor_coincidencia  # Sobrescribir la colonia
        df.loc[idx, 'geometría'] = df_colonias.iloc[idx_colonia]['geometry']  # Agregar la geometría
        df_actualizado = pd.concat([df_actualizado, df.loc[[idx]]])  # Añadir la fila actualizada
        print(f"Fila {idx}: Reemplazando '{colonia}' por '{mejor_coincidencia}' con {score}% de similitud")
    else:
        print(f"Fila {idx}: No se encontró coincidencia suficiente para '{colonia}' (Similitud: {score}%)")

# Guardar el DataFrame actualizado en un archivo CSV
df_actualizado.to_csv('df_con_colonias_actualizadas.csv', index=False)

print("DataFrame actualizado y guardado como 'df_con_colonias_actualizadas.csv'")


Fila 0: Reemplazando 'colonia bosques de la huasteca' por 'bosques de la huasteca' con 100% de similitud
Fila 1: Reemplazando 'colonia vista hermosa' por 'vista hermosa' con 100% de similitud
Fila 2: Reemplazando 'col potrero anáhuac' por 'potrero anahuac' con 97% de similitud
Fila 3: Reemplazando 'colonia satélite acueducto monterrey' por 'satelite acueducto' con 76% de similitud
Fila 4: Reemplazando 'colonia arboledas de la silla guadalupe' por 'arboledas de la silla' con 81% de similitud
Fila 5: Reemplazando 'colonia villas del mirador' por 'mirador de la silla' con 86% de similitud
Fila 9: No se encontró coincidencia suficiente para 'col hgo jal' (Similitud: 53%)
Fila 79: No se encontró coincidencia suficiente para 'colonia ciudad ideal danielcarrillo' (Similitud: 60%)
Fila 148: Reemplazando 'cervera del río' por 'cerrada del roble' con 77% de similitud
Fila 152: No se encontró coincidencia suficiente para 'burger king' (Similitud: 52%)
Fila 167: No se encontró coincidencia suficie